In [3]:
import requests
import pandas as pd
import json

In [4]:
def getAPIData(pages,
    username = 'imunizacao_public',
    password = 'qlto5t&7r_@+#Tlstigi',
    url_firstpage = 'https://imunizacao-es.saude.gov.br/_search?scroll=1m',
    url_nextpages = 'https://imunizacao-es.saude.gov.br/_search/scroll'
    ):

    '''
    Função para extrair todo os dados de vacinação da API.
    Para mais detalhes em relação à execução da chamada da API confira o manual contido dentro da pasta others.

    A função retorna o dataframe completo mais o status das suas respectivas respostas de consulta.
    Consulta pode demorar bastante, tempo estimado de 10 páginas por minuto, dependendo do processador.
    '''

    responses = []
    scroll_ids = []

    # Chamada da primeira página.
    response_pag1 = requests.post(url_firstpage, auth=(username, password), json={"size": 10000})
    
    print(f'Page 1 - Status Response: {response_pag1.status_code}')

    responses.append(response_pag1.status_code)

    # Normalização do arquivo em json e tranferência para um Dataframe.
    data = response_pag1.json()
    df = pd.json_normalize(data, record_path=['hits','hits'])

    # Para as próximas chamadas devemos gerar o scroll id para inserir no corpo da chamada da API
    scroll_id = data['_scroll_id']

    items = df.shape[0]

    # Loop while para conferir os conteúdos das demais páginas onde pag representa o número da página
    pag = 1
    while items == 10000 and pag < pages:

        pag+=1

        try:
            # Chamada das demais páginas.
            response_pagN = requests.post(url_nextpages, auth=(username, password), json={"scroll_id":scroll_id,"scroll": "1m"})
    
            if pag%10 == 0:
                print(f'Page {pag} - Status Response: {response_pagN.status_code}')
    
            responses.append(response_pagN.status_code)
    
            # Normalização do arquivo em json e tranferência para um Dataframe.
            dataN = response_pagN.json()
            dfN = pd.json_normalize(dataN, record_path=['hits','hits'])
    
            # Atribuição ao novo valor de items para verificação se a página contém mais do que o número máximo de registros (10000)
            items = dfN.shape[0]
    
            # Concatenação dos dados extraídos das seguintes leituras da API
            df = pd.concat([df,dfN],ignore_index=True)

        except MemoryError:
            print(f'At Page {pag} there was not enough memory to execute the task - Status Response: {response_pagN.status_code}\nThe scroll_id has been added to the list to a future attempt')
            scroll_ids.append(scroll_id)

        except ConnectionError:
            print('No Internet Connection to Perform the API Call.')

        
        scroll_id = dataN['_scroll_id']


    print(f'{pag} Pages have been loaded successfully!')

    print(f'The following responses status have been retrieved: {set(responses)}')

    print(f'Final Dataset Shape:\nRows: {df.shape[0]}\nColumns: {df.shape[1]}')
    
    return df, responses, scroll_ids
        

In [5]:
df_vac, responses, scroll_ids = getAPIData(10)

Page 1 - Status Response: 200
Page 10 - Status Response: 200
10 Pages have been loaded successfully!
The following responses status have been retrieved: {200}
Final Dataset Shape:
Rows: 100000
Columns: 46


In [7]:
df_vac.head(2)

,_index,_type,_id,_score,_source.estalecimento_noFantasia,_source.vacina_lote,_source.estabelecimento_municipio_codigo,_source.estabelecimento_valor,_source.vacina_nome,_source.ds_condicao_maternal,...,_source.paciente_endereco_nmMunicipio,_source.estabelecimento_municipio_nome,_source.vacina_codigo,_source.paciente_enumSexoBiologico,_source.dt_deleted,_source.co_condicao_maternal,_source.vacina_grupoAtendimento_nome,_source.paciente_racaCor_valor,_source.vacina_dataAplicacao,_source.data_importacao_rnds
0,desc-imunizacao-v5,_doc,458e6f91-c82c-4cf1-98b2-07a4fe71187a-i0b0,1.0,UAPS VIVIANE BENEVIDES,210524,230440,2482142,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,Nenhuma,...,FORTALEZA,FORTALEZA,86,M,None,1.0,Outros,PARDA,2022-10-31T00:00:00.000Z,2022-11-03T03:57:25.000Z
1,desc-imunizacao-v5,_doc,ed95af5c-12ba-47b2-a50d-6b9150055de6-i0b0,1.0,UNIDADE BASICA PONCE DE LEAO,208F21A,130140,2016222,COVID-19 JANSSEN - Ad26.COV2.S,Nenhuma,...,EIRUNEPE,EIRUNEPE,88,F,None,1.0,Pessoas de 18 a 64 anos,SEM INFORMACAO,2022-07-28T00:00:00.000Z,2022-11-02T19:19:46.000Z


In [16]:
for column in df_vac.columns:
    print(column)

_index
_type
_id
_score
_source.estalecimento_noFantasia
_source.vacina_lote
_source.estabelecimento_municipio_codigo
_source.estabelecimento_valor
_source.vacina_nome
_source.ds_condicao_maternal
_source.paciente_endereco_coPais
_source.@version
_source.document_id
_source.paciente_nacionalidade_enumNacionalidade
_source.vacina_categoria_codigo
_source.vacina_fabricante_referencia
_source.paciente_id
_source.paciente_idade
_source.vacina_descricao_dose
_source.paciente_endereco_coIbgeMunicipio
_source.vacina_grupoAtendimento_codigo
_source.data_importacao_datalake
_source.paciente_racaCor_codigo
_source.estabelecimento_uf
_source.estabelecimento_razaoSocial
_source.vacina_numDose
_source.@timestamp
_source.sistema_origem
_source.paciente_dataNascimento
_source.paciente_endereco_uf
_source.vacina_fabricante_nome
_source.paciente_endereco_cep
_source.id_sistema_origem
_source.status
_source.paciente_endereco_nmPais
_source.vacina_categoria_nome
_source.paciente_endereco_nmMunicipio
_sou

In [37]:
all_columns = '''_index
_type
_id
_score
_source.estalecimento_noFantasia
_source.vacina_lote
_source.estabelecimento_municipio_codigo
_source.estabelecimento_valor
_source.vacina_nome
_source.ds_condicao_maternal
_source.paciente_endereco_coPais
_source.@version
_source.document_id
_source.paciente_nacionalidade_enumNacionalidade
_source.vacina_categoria_codigo
_source.vacina_fabricante_referencia
_source.paciente_id
_source.paciente_idade
_source.vacina_descricao_dose
_source.paciente_endereco_coIbgeMunicipio
_source.vacina_grupoAtendimento_codigo
_source.data_importacao_datalake
_source.paciente_racaCor_codigo
_source.estabelecimento_uf
_source.estabelecimento_razaoSocial
_source.vacina_numDose
_source.@timestamp
_source.sistema_origem
_source.paciente_dataNascimento
_source.paciente_endereco_uf
_source.vacina_fabricante_nome
_source.paciente_endereco_cep
_source.id_sistema_origem
_source.status
_source.paciente_endereco_nmPais
_source.vacina_categoria_nome
_source.paciente_endereco_nmMunicipio
_source.estabelecimento_municipio_nome
_source.vacina_codigo
_source.paciente_enumSexoBiologico
_source.dt_deleted
_source.co_condicao_maternal
_source.vacina_grupoAtendimento_nome
_source.paciente_racaCor_valor
_source.vacina_dataAplicacao
_source.data_importacao_rnds
'''
all_columns = all_columns.split('\n')
all_columns.pop(-1)


columns = '''document_id
paciente_id
paciente_idade
paciente_dataNascimento
paciente_enumSexoBioIogico
paciente_racaCor_codigo
paciente_racaCor_valor
paciente_endereco_colbgeMunicipio
paciente_endereco_coPais
paciente_endereco_nmMunicipio
paciente_endereco_nmPais
paciente_endereco_cep
paciente_nacionalidade_enumNacionalidade
estabelecimento_valor
estabelecimento_razaoSocial
estalecimento_noFantasia
estabelecimento_municipio_codigo
estabelecimento_municipio_nome
estabelecimento_uf
vacina_grupo atendimento code
vacina_grupo_atendimento_nome
vacina_categoria_code
vacina_categoria_nome
vacina_lote
vacina_fabricante nome
vacina_fabricante referencia
vacina_dataAplicacao
vacina_descricao dose
vacina_codigo
vacina_nome
sistema_origem
'''
columns = columns.split('\n')
columns.pop(-1)
for i in range(len(columns)):
    columns[i] = '_source.'+columns[i]

In [40]:
columns_toRemove = []

for column in all_columns:
    if column not in columns:
        columns_toRemove.append(column)

# Deve ser 14.
print(len(columns_toRemove))

23


In [39]:
columns_toRemove

['_index',
 '_type',
 '_id',
 '_score',
 '_source.ds_condicao_maternal',
 '_source.@version',
 '_source.vacina_categoria_codigo',
 '_source.vacina_fabricante_referencia',
 '_source.vacina_descricao_dose',
 '_source.paciente_endereco_coIbgeMunicipio',
 '_source.vacina_grupoAtendimento_codigo',
 '_source.data_importacao_datalake',
 '_source.vacina_numDose',
 '_source.@timestamp',
 '_source.paciente_endereco_uf',
 '_source.vacina_fabricante_nome',
 '_source.id_sistema_origem',
 '_source.status',
 '_source.paciente_enumSexoBiologico',
 '_source.dt_deleted',
 '_source.co_condicao_maternal',
 '_source.vacina_grupoAtendimento_nome',
 '_source.data_importacao_rnds']

In [13]:
file_url = 'https://data.brasil.io/dataset/covid19/caso_full.csv.gz'

file = '../data/caso_full.rar'

response = requests.get(file_url)

with open(file, "wb") as casos:
    casos.write(response.content)

In [37]:
df_cases = pd.read_csv('../data/caso_full.csv',sep=',')

In [39]:
df_cases.head()

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,Rio Branco,1200401.0,2020-03-17,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-17,0.0,0,1,city,AC,3,0
1,NaN,12.0,2020-03-17,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0,0,1,state,AC,3,0
2,Rio Branco,1200401.0,2020-03-18,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-18,0.0,0,2,city,AC,0,0
3,NaN,12.0,2020-03-18,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0,0,2,state,AC,0,0
4,Rio Branco,1200401.0,2020-03-19,202012,413418.0,407319.0,False,False,4,0.96754,2020-03-19,0.0,0,3,city,AC,1,0
